In [8]:
import psycopg2
try:
    connection = psycopg2.connect(
        host = 'osu-smartpong-db.coppbwdukw7p.us-east-2.rds.amazonaws.com',
        port = 5432,
        user = 'smartpong',
        password = 'jvg08tgCZhWBnlEa2Qj9',
        database = 'smartpong_db'
        )
except:
    print('unable to connect')

cursor=connection.cursor()

In [19]:
try:
    cursor.execute("""CREATE TABLE testTable(
        episode_num real PRIMARY KEY,
        value real)""")
except:
    print('table creation failed')
    
connection.commit()

In [20]:
import pandas as pd

sql = """
SELECT "table_name","column_name", "data_type", "table_schema"
FROM INFORMATION_SCHEMA.COLUMNS
WHERE "table_schema" = 'public'
ORDER BY table_name  
"""
pd.read_sql(sql, con=connection)

,table_name,column_name,data_type,table_schema
0,testtable,episode_num,real,public
1,testtable,value,real,public


In [21]:
with open('20_row_db_test.txt', 'r') as row:
    cursor.copy_from(row, 'testTable', sep=' ')
connection.commit()

In [22]:
sql2 = """
SELECT * FROM testTable
"""
pd.read_sql(sql2, con=connection)

,episode_num,value
0,0.0,-17.0
1,1.0,-19.0
2,2.0,-16.0
3,3.0,-14.0
4,4.0,-20.0
5,5.0,-19.0
6,6.0,-19.0
7,7.0,-16.0
8,8.0,-17.0
9,9.0,-16.0


In [ ]:
if(connection):
    cursor.close()
    connection.close()

now lets test for storing of pickled objects, first with a small list then save.p, found in first/model of the code repository
from http://blog.cameronleger.com/2011/05/31/python-example-pickling-things-into-mysql-databases/

In [30]:
import _pickle as pickle

## Create a semi-complex list to pickle
listToPickle = [(10, 10), (20, 10.0), (1.0, 2.0)]

## Pickle the list into a string
pickledList = pickle.dumps(listToPickle)

# same as above
try:
    connection = psycopg2.connect(
        host = 'osu-smartpong-db.coppbwdukw7p.us-east-2.rds.amazonaws.com',
        port = 5432,
        user = 'smartpong',
        password = 'jvg08tgCZhWBnlEa2Qj9',
        database = 'smartpong_db'
        )
except:
    print('unable to connect')

cursor=connection.cursor()

pd.read_sql(sql, con=connection)


,table_name,column_name,data_type,table_schema
0,testtable,episode_num,real,public
1,testtable,value,real,public


In [42]:
try:
    cursor.execute("""CREATE TABLE testPickle(
        id int PRIMARY KEY,
        name varchar(20),
        list bytea)""")
except:
    print('table creation failed')
    
pd.read_sql(sql, con=connection)
connection.commit()

In [43]:
 ## Add the information to the database table pickleTest
cursor.execute("""INSERT INTO testPickle VALUES (1, 'pickletest', %s)""", (pickledList, ))
connection.commit()

In [48]:
## read what we added
sql3 = """
SELECT * FROM testPickle
"""
pd.read_sql(sql3, con=connection)

,id,name,list
0,1,pickletest,"[b'\x80', b'\x03', b']', b'q', b'\x00', b'(', ..."


In [49]:
## Select what we just added
cursor.execute("""SELECT list FROM testPickle WHERE id = 1""")

## Dump the results to a string
rows = cursor.fetchall()

print(rows)

[(<memory at 0x0000021784A58408>,)]


In [52]:
## Get the results (added print statements to see what is happening)
for each in rows:
    print(each)
    ## The result is also in a tuple
    for pickledStoredList in each:
        print(pickledStoredList)
        ## Unpickle the stored string
        unpickledList = pickle.loads(pickledStoredList)
        print(unpickledList)

(<memory at 0x0000021784A58408>,)
[(10, 10), (20, 10.0), (1.0, 2.0)]


In [53]:
## now lets try with save.p, start with a commit to give a starting point
connection.commit()
## create the model like we would have at the end of a run
model = pickle.load(open('save.p', 'rb'))
## ensure the model looks correct
print(model)


{'W1': array([[ 0.00986305,  0.01918138, -0.00313199, ..., -0.01376265,
         0.01535608,  0.00253256],
       [ 0.00385199,  0.01159074,  0.00542042, ..., -0.00051124,
        -0.01280302, -0.00149491],
       [-0.00265701, -0.01828055,  0.01837293, ...,  0.00580771,
        -0.00133841,  0.00547078],
       ...,
       [-0.00151471, -0.01705434, -0.00998575, ...,  0.00595084,
        -0.01750963, -0.0131777 ],
       [-0.0122187 ,  0.03356755,  0.00031973, ...,  0.0110237 ,
         0.02433764,  0.00318426],
       [ 0.01251938,  0.0086053 , -0.00484523, ..., -0.00468177,
        -0.00273522,  0.00399169]]), 'W2': array([-0.05176747,  0.01042403,  0.06809449, -0.05495598,  0.06147652,
        0.04889752,  0.02549399, -0.14199327,  0.00875722, -0.0611561 ,
        0.06237162, -0.0676683 , -0.14531787, -0.08077788,  0.18096023,
        0.06802402,  0.04962563, -0.07516429, -0.15192587, -0.06492728,
        0.04385191, -0.01838174,  0.03493977,  0.12697728, -0.03480043,
        0.017

In [54]:
## now pickle like at the end of the program
pickled_model = pickle.dumps(model)

## establish connection (this should not need to be done every time in
## the actual program but done here due to the out of order code execution
## juyptier allows you could start from this code block ((actually the testPickle
# table is resused so maybe not))
# same as above
try:
    connection = psycopg2.connect(
        host = 'osu-smartpong-db.coppbwdukw7p.us-east-2.rds.amazonaws.com',
        port = 5432,
        user = 'smartpong',
        password = 'jvg08tgCZhWBnlEa2Qj9',
        database = 'smartpong_db'
        )
except:
    print('unable to connect')

cursor=connection.cursor()

pd.read_sql(sql, con=connection)

,table_name,column_name,data_type,table_schema
0,testpickle,name,character varying,public
1,testpickle,list,bytea,public
2,testpickle,id,integer,public
3,testtable,episode_num,real,public
4,testtable,value,real,public


In [55]:
 ## Add the information to the database table pickleTest
cursor.execute("""INSERT INTO testPickle VALUES (2, 'modeltest', %s)""", (pickled_model, ))
connection.commit()
## read what we just added
pd.read_sql(sql3, con=connection)

,id,name,list
0,1,pickletest,"[b'\x80', b'\x03', b']', b'q', b'\x00', b'(', ..."
1,2,modeltest,"[b'\x80', b'\x03', b'}', b'q', b'\x00', b'(', ..."


In [56]:
## Select what we just added
cursor.execute("""SELECT list FROM testPickle WHERE id = 2""")

## Dump the results to a string
rows = cursor.fetchall()

print(rows)

[(<memory at 0x0000021784A58A08>,)]


In [59]:
## Get the results (added print statements to see what is happening)
for each in rows:
    print(each)
    ## The result is also in a tuple
    for pickledStoredModel in each:
        print(pickledStoredModel)
        ## Unpickle the stored string
        unpickledModel = pickle.loads(pickledStoredModel)
        print(unpickledModel)
        ## compare with original
        print(model)

(<memory at 0x0000021784A58A08>,)
{'W1': array([[ 0.00986305,  0.01918138, -0.00313199, ..., -0.01376265,
         0.01535608,  0.00253256],
       [ 0.00385199,  0.01159074,  0.00542042, ..., -0.00051124,
        -0.01280302, -0.00149491],
       [-0.00265701, -0.01828055,  0.01837293, ...,  0.00580771,
        -0.00133841,  0.00547078],
       ...,
       [-0.00151471, -0.01705434, -0.00998575, ...,  0.00595084,
        -0.01750963, -0.0131777 ],
       [-0.0122187 ,  0.03356755,  0.00031973, ...,  0.0110237 ,
         0.02433764,  0.00318426],
       [ 0.01251938,  0.0086053 , -0.00484523, ..., -0.00468177,
        -0.00273522,  0.00399169]]), 'W2': array([-0.05176747,  0.01042403,  0.06809449, -0.05495598,  0.06147652,
        0.04889752,  0.02549399, -0.14199327,  0.00875722, -0.0611561 ,
        0.06237162, -0.0676683 , -0.14531787, -0.08077788,  0.18096023,
        0.06802402,  0.04962563, -0.07516429, -0.15192587, -0.06492728,
        0.04385191, -0.01838174,  0.03493977,  0.12

In [61]:
## more function testing, easier to get 'live' feedback from this notebook
try:
    connection = psycopg2.connect(
        host = 'osu-smartpong-db.coppbwdukw7p.us-east-2.rds.amazonaws.com',
        port = 5432,
        user = 'smartpong',
        password = 'jvg08tgCZhWBnlEa2Qj9',
        database = 'smartpong_db'
        )
except:
    print('unable to connect')

cursor=connection.cursor()



In [62]:
## tables will now look more like the planned schema
## this will be like the models table save for the pickle data (see above)

try:
    cursor.execute("""CREATE TABLE testModel(
        name varchar(32) PRIMARY KEY,
        node_num int,
        batch_size int,
        learning_rate real,
        gamma real,
        decay real)""")
except:
    print('table creation failed')
    
connection.commit()

In [68]:
## same hyperparams list that would be returned from the interface
## hyperparameters
H = 300  # number of hidden layer neurons
batch_size = 7  # every how many episodes to do a param update?
learning_rate = 5e-4
gamma = 0.99  # discount factor for reward
decay_rate = 0.99  # decay factor for RMSProp leaky sum of grad^2
## make list
hyper_params = [H, batch_size, learning_rate, gamma, decay_rate]
model_name = 'test_model'

## now try to insert information
sqlm = """INSERT INTO testModel
VALUES (%s, %s, %s, %s, %s, %s)"""
data = (model_name, hyper_params[0], hyper_params[1], hyper_params[2], hyper_params[3], hyper_params[4])
cursor.execute(sqlm, data)
connection.commit()

In [73]:
## try to retrieve data
sqls = """SELECT node_num, batch_size, learning_rate, gamma, decay
    FROM testModel
    WHERE name = %s"""
input_name = 'test_model'
datas = (input_name, )
cursor.execute(sqls, datas)
## number of rows returned, should be 1
print(cursor.rowcount)
row = cursor.fetchone()
print(row)

1
(300, 7, 0.0005, 0.99, 0.99)
